In [2]:
# /Users/kd/Documents/github_code/data-engineering/01-spark/data/ml-25m

In [3]:
from pyspark.sql import SparkSession

In [4]:
MAX_MEMORY= '5g'
spark = SparkSession.builder.appName("movie-recommendation")\
    .config("spark.executor.memory", MAX_MEMORY)\
    .config("spark.driver.memory", MAX_MEMORY )\
    .getOrCreate()

22/07/31 16:39:05 WARN Utils: Your hostname, kds-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.20.10.13 instead (on interface en0)
22/07/31 16:39:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/31 16:39:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
ratings_file = "/Users/kd/Documents/github_code/data-engineering/01-spark/data/ml-25m/ratings.csv"
ratings_df = spark.read.csv(f"file:///{ratings_file}", inferSchema=True, header=True)

In [6]:
ratings_df.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
|     1|   1088|   4.0|1147868495|
|     1|   1175|   3.5|1147868826|
|     1|   1217|   3.5|1147878326|
|     1|   1237|   5.0|1147868839|
|     1|   1250|   4.0|1147868414|
|     1|   1260|   3.5|1147877857|
|     1|   1653|   4.0|1147868097|
|     1|   2011|   2.5|1147868079|
|     1|   2012|   2.5|1147868068|
|     1|   2068|   2.5|1147869044|
|     1|   2161|   3.5|1147868609|
|     1|   2351|   4.5|1147877957|
|     1|   2573|   4.0|1147878923|
|     1|   2632|   5.0|1147878248|
|     1|   2692|   5.0|1147869100|
+------+-------+------+----------+
only showing top 20 rows



In [7]:
ratings_df = ratings_df.select(["userId","movieId","rating"])

In [8]:
ratings_df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



In [9]:
ratings_df.select("rating").describe().show()

+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|          25000095|
|   mean| 3.533854451353085|
| stddev|1.0607439611423508|
|    min|               0.5|
|    max|               5.0|
+-------+------------------+



In [10]:
train_df, test_df = ratings_df.randomSplit([0.8,0.2])

In [11]:
from pyspark.ml.recommendation import ALS

In [12]:
als = ALS(
    maxIter=5,
    regParam=0.1,
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    coldStartStrategy="drop"
)

In [13]:
model = als.fit(train_df)

22/07/31 16:39:27 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/07/31 16:39:27 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/07/31 16:39:27 WARN InstanceBuilder$JavaBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
22/07/31 16:39:28 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [14]:
predictions = model.transform(test_df)

In [15]:
predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|    76|   1342|   3.5| 2.9933548|
|   101|   8638|   5.0| 3.5224414|
|   296|   1580|   3.0|  2.750238|
|   322|   1645|   4.0| 3.5329566|
|   368|   1580|   3.5| 3.6636012|
|   460|  44022|   3.0| 3.8867457|
|   472|   3918|   3.0| 2.3742616|
|   481|   1580|   4.0| 3.5850673|
|   501|   1580|   5.0| 3.8954766|
|   513|  44022|   5.0| 4.3556004|
|   588|   1645|   2.5| 2.7095046|
|   596|   1580|   3.0| 3.5189588|
|   597|   1580|   4.0|   3.72535|
|   606|   5803|   4.5| 3.6512556|
|   606| 160563|   4.0| 3.9676142|
|   626|   2866|   3.0| 3.4082007|
|   626|   6658|   3.5| 2.8752565|
|   633|   1591|   5.0|  3.306498|
|   744|  44022|   3.5| 3.6308057|
|   772|    471|   4.0| 3.3049536|
+------+-------+------+----------+
only showing top 20 rows



In [16]:
predictions.select("rating","prediction").describe().show()

+-------+-----------------+------------------+
|summary|           rating|        prediction|
+-------+-----------------+------------------+
|  count|          4995812|           4995812|
|   mean|3.533921412575173|3.3915717886959422|
| stddev|1.060619136936604|0.6353947127569068|
|    min|              0.5|        -1.2873489|
|    max|              5.0|          6.834832|
+-------+-----------------+------------------+



In [17]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

In [18]:
rmse = evaluator.evaluate(predictions)

In [19]:
print(rmse)

0.8162247169737259


In [20]:
model.recommendForAllUsers(3).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    26|[{203086, 5.6749}...|
|    27|[{194714, 5.91885...|
|    28|[{194434, 7.97853...|
|    31|[{194434, 4.00649...|
|    34|[{194434, 6.32072...|
|    44|[{205277, 7.07942...|
|    53|[{194334, 6.91518...|
|    65|[{205277, 7.19286...|
|    76|[{194434, 6.58554...|
|    78|[{194434, 7.57404...|
|    81|[{200930, 5.07900...|
|    85|[{197927, 5.83198...|
|   101|[{203086, 5.47180...|
|   103|[{194434, 6.65387...|
|   108|[{194434, 5.73014...|
|   115|[{194434, 6.43967...|
|   126|[{203086, 6.66958...|
|   133|[{177209, 6.25674...|
|   137|[{177209, 6.15170...|
|   148|[{194434, 6.23394...|
+------+--------------------+
only showing top 20 rows



In [21]:
model.recommendForAllItems(3).show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     12|[{87426, 5.124295...|
|     26|[{105801, 5.19086...|
|     27|[{87426, 5.22027}...|
|     28|[{105801, 5.58734...|
|     31|[{87426, 5.218137...|
|     34|[{141970, 5.32036...|
|     44|[{87426, 5.106878...|
|     53|[{27587, 5.187121...|
|     65|[{93649, 4.741573...|
|     76|[{87426, 5.354525...|
|     78|[{142811, 4.68062...|
|     81|[{152146, 4.69532...|
|     85|[{105801, 4.95340...|
|    101|[{104135, 4.97959...|
|    103|[{96471, 5.012725...|
|    108|[{111212, 4.90138...|
|    115|[{67565, 5.710346...|
|    126|[{87426, 4.903103...|
|    133|[{67565, 4.433848...|
|    137|[{67565, 5.059546...|
+-------+--------------------+
only showing top 20 rows



In [22]:
from pyspark.sql.types import IntegerType

user_list = [65, 70, 81]
users_df = spark.createDataFrame(user_list, IntegerType()).toDF('userId')

users_df.show()

+------+
|userId|
+------+
|    65|
|    70|
|    81|
+------+



In [23]:
user_recs = model.recommendForUserSubset(users_df, 5)

In [24]:
movies_list = user_recs.collect()[0].recommendations

In [25]:
recs_df = spark.createDataFrame(movies_list)
recs_df.show()

+-------+------------------+
|movieId|            rating|
+-------+------------------+
| 205277| 7.192861557006836|
| 203086| 6.727421760559082|
| 196717| 6.513143062591553|
| 159761|6.4345221519470215|
| 194434| 6.428370475769043|
+-------+------------------+



In [26]:
movies_file = "/Users/kd/Documents/github_code/data-engineering/01-spark/data/ml-25m/movies.csv"
movies_df = spark.read.csv(f"file:///{movies_file}", inferSchema=True, header=True)

In [27]:
movies_df.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [28]:
recs_df.createOrReplaceTempView("recommendations")
movies_df.createOrReplaceTempView("movies")

In [29]:
query = """
SELECT *
FROM
    movies JOIN recommendations
    ON movies.movieId = recommendations.movieId
ORDER BY
    rating desc
"""
recommended_movies = spark.sql(query)
recommended_movies.show()

+-------+--------------------+--------------------+-------+------------------+
|movieId|               title|              genres|movieId|            rating|
+-------+--------------------+--------------------+-------+------------------+
| 205277|   Inside Out (1991)|Comedy|Drama|Romance| 205277| 7.192861557006836|
| 203086|Truth and Justice...|               Drama| 203086| 6.727421760559082|
| 196717|Bernard and the G...|Comedy|Drama|Fantasy| 196717| 6.513143062591553|
| 159761|         Loot (1970)|        Comedy|Crime| 159761|6.4345221519470215|
| 194434|   Adrenaline (1990)|  (no genres listed)| 194434| 6.428370475769043|
+-------+--------------------+--------------------+-------+------------------+



In [30]:
def get_recommendations(user_id, num_recs):
    users_df = spark.createDataFrame([user_id], IntegerType()).toDF("userId")
    user_recs_df = model.recommendForUserSubset(users_df, num_recs)
    
    recs_list = user_recs_df.collect()[0].recommendations
    recs_df = spark.createDataFrame(recs_list)
    recommended_movies = spark.sql(query)
    return recommended_movies

In [31]:
recs = get_recommendations(456, 10)

In [32]:
recs.toPandas()

,movieId,title,genres,movieId,rating
0,205277,Inside Out (1991),Comedy|Drama|Romance,205277,7.192862
1,203086,Truth and Justice (2019),Drama,203086,6.727422
2,196717,Bernard and the Genie (1991),Comedy|Drama|Fantasy,196717,6.513143
3,159761,Loot (1970),Comedy|Crime,159761,6.434522
4,194434,Adrenaline (1990),(no genres listed),194434,6.428370
